In [1]:
import numpy as np
import mlflow

from mlflow.models import infer_signature
import torch
from torch import nn
import pandas as  pd

In [2]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

net = nn.Linear(10, 1)
loss_function = nn.L1Loss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)

X = torch.randn(100, 10)
y = torch.randn(100, 1)

print(X.shape, y.shape)

epochs = 5
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = net(X)

    loss = loss_function(outputs, y)
    loss.backward()

    optimizer.step()

with mlflow.start_run() as run:
    signature = infer_signature(X.numpy(), net(X).detach().numpy())
    model_info = mlflow.pytorch.log_model(
        pytorch_model = net, 
        artifact_path = "pytorch model", 
        signature=signature, 
        input_example=X.numpy(),
        registered_model_name="pytorch_model"
        )

    pytorch_pyfunc = mlflow.pyfunc.load_model(model_uri=model_info.model_uri)

    X_test = torch.randn(20, 10).numpy()
    predictions = pytorch_pyfunc.predict(X_test)
    print(predictions)

    
    eval_data = pd.DataFrame(X.numpy())
    eval_data = pd.DataFrame(y.numpy())
    print(eval_data)
    eval_data.columns = ["label"]
    eval_data["predictions"] = net(X).detach().numpy()
    print(eval_data.shape)
    
    
    results = mlflow.evaluate(
        data=eval_data,
        model_type="regressor",
        targets= "label",
        predictions="predictions",
        evaluators = ["default"]
    )

print(f"metrics:\n{results.metrics}")
print(f"artifacts:\n{results.artifacts}")

torch.Size([100, 10]) torch.Size([100, 1])


Successfully registered model 'pytorch_model'.
2024/09/19 01:01:08 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: pytorch_model, version 1
Created version '1' of model 'pytorch_model'.


[[ 0.04673885]
 [ 0.56937176]
 [ 1.4875739 ]
 [ 0.5236491 ]
 [ 0.38019693]
 [ 0.9018043 ]
 [ 0.9595364 ]
 [-1.166437  ]
 [ 0.337833  ]
 [ 0.79510224]
 [ 0.15641308]
 [ 0.3382789 ]
 [ 0.15541911]
 [-0.53901947]
 [ 0.07947662]
 [ 0.658213  ]
 [-0.3296793 ]
 [ 0.02152778]
 [ 1.2062057 ]
 [ 0.65615463]]
           0
0   1.578726
1  -0.014986
2   0.471563
3   0.216750
4  -0.223432
..       ...
95 -0.543844
96  1.300391
97  0.343706
98 -1.508931
99  0.635036

[100 rows x 1 columns]
(100, 2)


/Users/ilia/micromamba/envs/mlops-pmldl/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/09/19 01:01:09 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/09/19 01:01:09 WARNING mlflow.models.evaluation.default_evaluator: SHAP or matplotlib package is not installed, so model explainability insights will not be logged.
2024/09/19 01:01:09 INFO mlflow.tracking._tracking_service.client: 🏃 View run bouncy-bird-959 at: http://127.0.0.1:8080/#/experiments/0/runs/869c98ab36a742fca9ebe3bbdfb729e3.
2024/09/19 01:01:09 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0.


metrics:
{'example_count': 100, 'mean_absolute_error': 1.0618036, 'mean_squared_error': 1.6085092, 'root_mean_squared_error': 1.2682701, 'sum_on_target': 7.603937869891524, 'mean_on_target': 0.07603937869891525, 'r2_score': -0.6416995525360107, 'max_error': 3.3447366, 'mean_absolute_percentage_error': 2.7787085}
artifacts:
{}


In [3]:
import mlflow.pyfunc
from mlflow import MlflowClient

run_id = "e389609f9f1b44678ea7fea020453f94"
model_artifact_path = "pytorch model"

model = mlflow.pyfunc.load_model(model_uri=f"runs:/{run_id}/{model_artifact_path}")

print(model.metadata)

# OR

model_name = "pytorch_model"
model_version = 1

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

print(model.metadata)

# OR

client = MlflowClient()
client.set_registered_model_alias(name = model_name, alias = "staging", version = "1")

model_name = "pytorch_model"
model_alias = "staging"

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}@{model_alias}")

print(model.metadata)

KeyboardInterrupt: 